In [1]:
# Import packages
import os, sys, glob, re, math, pickle
import phate, scprep, magic, meld
import graphtools as gt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time,random,datetime
import networkx as nx
import scvelo as scv
from sklearn import metrics
from sklearn import model_selection
from scipy import sparse
from scipy.stats import mannwhitneyu, tiecorrect, rankdata
from statsmodels.stats.multitest import multipletests
import scanpy as sc
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import SpectralClustering, OPTICS, cluster_optics_dbscan, AgglomerativeClustering
from bbknn import bbknn
import warnings
%matplotlib inline
%load_ext memory_profiler


In [2]:
import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [3]:
# settings
plt.rc('font', size = 8)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=8)
plt.rcParams['savefig.dpi']=600
sc.settings.verbosity=2
sc._settings.ScanpyConfig.n_jobs=-1

# reproducibility
rs = np.random.seed(42)

# utils
def mwu(X,Y,gene_names,correction=None,debug=False) :
    '''
    Benjamini-Hochberg correction implemented. Can change to Bonferonni
    gene_names (list)
    if X,Y single gene expression array, input x.reshape(-1,1), y.reshape(-1,1)
    NOTE: get zeros sometimes because difference (p-value is so small)
    '''
    p=pd.DataFrame()
    print('Mann-Whitney U w/Benjamini/Hochberg correction\n')
    start = time.time()
    for i,g in enumerate(gene_names) :
        if i==np.round(np.quantile(np.arange(len(gene_names)),0.25)) :
            print('... 25% completed in {:.2f}-s'.format(time.time()-start))
        elif i==np.round(np.quantile(np.arange(len(gene_names)),0.5)) :
            print('... 50% completed in {:.2f}-s'.format(time.time()-start))
        elif i==np.round(np.quantile(np.arange(len(gene_names)),0.75)) :
            print('... 75% completed in {:.2f}-s'.format(time.time()-start))
        p.loc[i,'Gene']=g
        if (tiecorrect(rankdata(np.concatenate((np.asarray(X[:,i]),np.asarray(Y[:,i])))))==0) :
            if debug :
                print('P-value not calculable for {}'.format(g))
            p.loc[i,'pval']=np.nan
        else :
            _,p.loc[i,'pval']=mannwhitneyu(X[:,i],Y[:,i]) # continuity correction is True
    print('... mwu computed in {:.2f}-s\n'.format(time.time() - start))
    # ignore NaNs, since can't do a comparison on these (change numbers for correction)
    p_corrected = p.loc[p['pval'].notna(),:]
    new_pvals = multipletests(p_corrected['pval'],method='fdr_bh')
    p_corrected['pval_corrected'] = new_pvals[1]
    return p_corrected

def log2aveFC(X,Y,gene_names,AnnData=None) :
    '''not sensitivity to directionality due to subtraction
    X and Y full arrays, subsetting performed here
    `gene_names` (list): reduced list of genes to calc
    `adata` (sc.AnnData): to calculate reduced list. NOTE: assumes X,Y drawn from adata.var_names
    '''
    if not AnnData is None :
        g_idx = [i for i,g in enumerate(AnnData.var_names) if g in gene_names]
        fc=pd.DataFrame({'Gene':AnnData.var_names[g_idx],
                         'log2FC':np.log2(X[:,g_idx].mean(axis=0)) - np.log2(Y[:,g_idx].mean(axis=0))}) # returns NaN if negative value 
    else :
        fc=pd.DataFrame({'Gene':gene_names,
                         'log2FC':np.log2(X.mean(axis=0)) - np.log2(Y.mean(axis=0))})
    return fc

In [4]:
# fps
dfp = '/home/cl2292/project/SCA1_snRNAseq/Human/data/'
pfp = '/home/cl2292/project/SCA1_snRNAseq/Human/results_20230402/'
pdfp = '/home/cl2292/project/SCA1_snRNAseq/Human/data/processed/'
sc.settings.figdir = pfp

In [5]:
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_Ctrl_imp.h5ad' # for full, can maybe get away with ~300G
    %memit wt = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_SCA1_imp.h5ad' # for full, can maybe get away with ~300G
    %memit mut = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))

peak memory: 43321.00 MiB, increment: 43019.17 MiB
loaded @230502.16:33:12
took 38.93-s to load data
peak memory: 73642.80 MiB, increment: 30328.06 MiB
loaded @230502.16:34:02
took 49.71-s to load data


In [7]:
print(wt.obs.genotype.value_counts())
print(mut.obs.genotype.value_counts())

Ctrl    139780
Name: genotype, dtype: int64
SCA1    97776
Name: genotype, dtype: int64


In [8]:
wt1 = wt.obs.loc[wt.obs['genotype']=='Ctrl',:].sample(n=97776, replace=False).index.to_list()

wt2 = wt[wt.obs.index.isin(wt1), :]

In [9]:
print(wt2.obs.genotype.value_counts())
print(mut.obs.genotype.value_counts())

Ctrl    97776
Name: genotype, dtype: int64
SCA1    97776
Name: genotype, dtype: int64


In [11]:
print(mut.obs['sub4'].value_counts())
print('\n')
print(wt2.obs['sub4'].value_counts())

GC      80613
BG       4384
AS       3367
MLI2     1960
OL       1946
MLI1     1684
MG       1315
END       967
OPC       879
PER       254
PC        224
GoC       105
UBC        78
Name: sub4, dtype: int64


GC      86651
BG       2445
OL       2367
AS       1648
MLI1     1331
MLI2     1088
OPC       713
MG        638
PC        370
END       365
PER        73
GoC        44
UBC        43
Name: sub4, dtype: int64


In [12]:
# save data objects
wt2.write(os.path.join(pdfp,'230502_Ctrl_sampling.h5ad'))
#sample_mut.write(os.path.join(dfp,'211102_SCA1_sampling.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

saved @230502.17:09:31


In [18]:
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_Ctrl_sampling.h5ad' # for full, can maybe get away with ~300G
    %memit wt = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_SCA1_imp.h5ad' # for full, can maybe get away with ~300G
    %memit mut = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))

peak memory: 91374.45 MiB, increment: 29977.86 MiB
loaded @230502.18:19:42
took 28.57-s to load data
peak memory: 91423.40 MiB, increment: 30245.30 MiB
loaded @230502.18:20:12
took 29.67-s to load data


In [14]:
# subset data to reduce data size and memory requirement
obs='sub4'
wt = wt[(wt.obs[obs] == 'UBC')|(wt.obs[obs] == 'PC')|(wt.obs[obs] == 'MLI1')|(wt.obs[obs] == 'MLI2')|(wt.obs[obs] == 'GOC'), :]
mut = mut[(mut.obs[obs] == 'UBC')|(mut.obs[obs] == 'PC')|(mut.obs[obs] == 'MLI1')|(mut.obs[obs] == 'MLI2')|(mut.obs[obs] == 'GOC'), :]
# save data objects
wt.write(os.path.join(pdfp,'230502_Ctrl_sampling_UBC PC MLI1 MLI2 GOC.h5ad'))
mut.write(os.path.join(pdfp,'230502_SCA1_sampling_UBC PC MLI1 MLI2 GOC.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

In [17]:
# subset data to reduce data size and memory requirement
obs='sub4'
wt = wt[(wt.obs[obs] == 'AS')|(wt.obs[obs] == 'BG')|(wt.obs[obs] == 'OPC')|(wt.obs[obs] == 'OL')
        |(wt.obs[obs] == 'MG')|(wt.obs[obs] == 'PER')|(wt.obs[obs] == 'END'), :]
mut = mut[(mut.obs[obs] == 'AS')|(mut.obs[obs] == 'BG')|(mut.obs[obs] == 'OPC')|(mut.obs[obs] == 'OL')
          |(mut.obs[obs] == 'MG')|(mut.obs[obs] == 'PER')|(mut.obs[obs] == 'END'), :]
# save data objects
wt.write(os.path.join(pdfp,'230502_Ctrl_sampling_AS BG OPC OL MG PER END.h5ad'))
mut.write(os.path.join(pdfp,'230502_SCA1_sampling_AS BG OPC OL MG PER END.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

saved @230502.18:18:42


In [19]:
# subset data to reduce data size and memory requirement
obs='sub4'
wt = wt[(wt.obs[obs] == 'GC'), :]
mut = mut[(mut.obs[obs] == 'GC'), :]
# save data objects
wt.write(os.path.join(pdfp,'230502_Ctrl_sampling_GC.h5ad'))
mut.write(os.path.join(pdfp,'230502_SCA1_sampling_GC.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

saved @230502.18:21:34


In [5]:
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_Ctrl_sampling_GC.h5ad' # for full, can maybe get away with ~300G
    %memit wt = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_SCA1_sampling_GC.h5ad' # for full, can maybe get away with ~300G
    %memit mut = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))

peak memory: 26869.19 MiB, increment: 26568.38 MiB
loaded @230503.11:50:10
took 25.83-s to load data
peak memory: 51782.29 MiB, increment: 24913.08 MiB
loaded @230503.11:50:35
took 24.86-s to load data


In [6]:
wt1 = wt.obs.loc[wt.obs['genotype']=='Ctrl',:].sample(n=50000, replace=False).index.to_list()

wt2 = wt[wt.obs.index.isin(wt1), :]

mut1 = mut.obs.loc[mut.obs['genotype']=='SCA1',:].sample(n=50000, replace=False).index.to_list()

mut2 = mut[mut.obs.index.isin(mut1), :]

In [9]:
wt2.write(os.path.join(pdfp,'230503_Ctrl_sampling_GC50000.h5ad'))
mut2.write(os.path.join(pdfp,'230503_SCA1_sampling_GC50000.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

saved @230503.11:53:12


In [8]:
print(wt2.obs.sub4.value_counts())
print(mut2.obs.sub4.value_counts())

GC    50000
Name: sub4, dtype: int64
GC    50000
Name: sub4, dtype: int64
